## 6. WAV 파일 길이 - JSON 라벨링 간 매칭 검증

In [1]:
import os
import json
import librosa
import pandas as pd

# 데이터 디렉토리 설정
data_dir = "/home/ubuntu/data"

# 파일 정보를 저장할 딕셔너리
wav_durations = {}
json_durations = {}

# 고유 ID 추출을 위한 정규 표현식
import re
id_pattern = re.compile(r'_(\d+)(?:_\d+)?$')  # 파일명에서 숫자 ID 추출

# 데이터 디렉토리 순회하며 파일 정보 수집
for root, _, files in os.walk(data_dir):
    for file in files:
        # WAV 파일 처리
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            try:
                # librosa로 파일 로드하여 오디오 데이터 및 샘플링 레이트 획득
                y, sr = librosa.load(file_path, sr=None, mono=False)

                # 파일 길이 계산 (초 단위)
                duration = librosa.get_duration(y=y, sr=sr)

                # 파일명에서 고유 ID 추출
                match = id_pattern.search(os.path.splitext(file)[0])
                if match:
                    file_id = match.group(1)
                    # 파일 정보 저장
                    wav_durations[file_id] = {
                        "wav_duration": duration,
                        "wav_path": file_path,
                        "wav_file_name": file
                    }
                else:
                    print(f"⚠️ WAV 파일에서 ID를 추출할 수 없습니다: {file_path}")

            except Exception as e:
                print(f"❌ WAV 파일 처리 중 오류 발생: {file_path}, 오류: {e}")

        # JSON 파일 처리
        elif file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data_json = json.load(f)
                    # "audio" 섹션의 "duration" 값 가져오기
                    duration = data_json.get("audio", {}).get("duration", None)

                    # 파일명에서 고유 ID 추출
                    match = id_pattern.search(os.path.splitext(file)[0])
                    if match:
                        file_id = match.group(1)
                        # 파일 정보 저장
                        json_durations[file_id] = {
                            "json_duration": duration,
                            "json_path": file_path,
                            "json_file_name": file
                        }
                    else:
                        print(f"⚠️ JSON 파일에서 ID를 추출할 수 없습니다: {file_path}")

            except Exception as e:
                print(f"❌ JSON 파일 처리 중 오류 발생: {file_path}, 오류: {e}")

# WAV 파일과 JSON 파일 매칭하여 길이 비교
results = []

for file_id in json_durations:
    if file_id in wav_durations:
        wav_info = wav_durations[file_id]
        json_info = json_durations[file_id]

        wav_duration = wav_info["wav_duration"]
        json_duration = json_info["json_duration"]

        # JSON의 duration이 None이 아닌지 확인
        if json_duration is not None:
            # 오차 계산 (초 단위)
            duration_error = abs(wav_duration - json_duration)
            # 허용 오차 범위 (예: 0.1초)
            tolerance = 0.1
            is_matching = duration_error <= tolerance

            results.append({
                "file_id": file_id,
                "wav_file_name": wav_info["wav_file_name"],
                "json_file_name": json_info["json_file_name"],
                "wav_duration": wav_duration,
                "json_duration": json_duration,
                "duration_error": duration_error,
                "is_matching": is_matching,
                "wav_path": wav_info["wav_path"],
                "json_path": json_info["json_path"]
            })
        else:
            print(f"⚠️ JSON 파일에 'duration' 값이 없습니다: {json_info['json_path']}")
    else:
        print(f"⚠️ 매칭되는 WAV 파일이 없습니다: ID={file_id}, JSON 파일={json_info['json_file_name']}")

# 결과를 데이터프레임으로 변환
df_results = pd.DataFrame(results)

# 오차가 허용 범위를 벗어난 파일 필터링
mismatched_files = df_results[~df_results["is_matching"]]

print(f"\n총 파일 수: {len(df_results)}")
print(f"일치하는 파일 수: {len(df_results) - len(mismatched_files)}")
print(f"오차가 있는 파일 수: {len(mismatched_files)}")

# 오차가 있는 파일 목록 출력 (최대 20개만 표시)
if not mismatched_files.empty:
    print("\n⚠️ WAV 파일과 JSON 파일의 길이가 일치하지 않는 파일 목록 (최대 20개):")
    for idx, row in mismatched_files.head(20).iterrows():
        print(f"ID: {row['file_id']}, WAV 파일: {row['wav_file_name']}, WAV 길이: {row['wav_duration']:.2f}s, JSON 길이: {row['json_duration']:.2f}s, 오차: {row['duration_error']:.2f}s")
else:
    print("\n✅ 모든 파일의 길이가 일치합니다!")

# 필요에 따라 CSV로 저장 (선택 사항)
# df_results.to_csv("wav_json_duration_comparison.csv", index=False)


총 파일 수: 18048
일치하는 파일 수: 7
오차가 있는 파일 수: 18041

⚠️ WAV 파일과 JSON 파일의 길이가 일치하지 않는 파일 목록 (최대 20개):
ID: 39554, WAV 파일: 2.motorcycle_horn_39554_1.wav, WAV 길이: 4.82s, JSON 길이: 8.82s, 오차: 4.00s
ID: 83873, WAV 파일: 2.motorcycle_horn_83873_1.wav, WAV 길이: 13.87s, JSON 길이: 17.87s, 오차: 4.00s
ID: 83258, WAV 파일: 2.motorcycle_horn_83258_1.wav, WAV 길이: 1.71s, JSON 길이: 5.71s, 오차: 4.00s
ID: 83833, WAV 파일: 2.motorcycle_horn_83833_1.wav, WAV 길이: 9.26s, JSON 길이: 13.26s, 오차: 4.00s
ID: 84352, WAV 파일: 2.motorcycle_horn_84352_1.wav, WAV 길이: 1.71s, JSON 길이: 5.71s, 오차: 4.00s
ID: 35131, WAV 파일: 2.motorcycle_horn_35131_1.wav, WAV 길이: 18.00s, JSON 길이: 22.00s, 오차: 4.00s
ID: 13230, WAV 파일: 2.motorcycle_horn_13230_1.wav, WAV 길이: 1.03s, JSON 길이: 5.03s, 오차: 4.00s
ID: 84697, WAV 파일: 2.motorcycle_horn_84697_1.wav, WAV 길이: 1.27s, JSON 길이: 5.27s, 오차: 4.00s
ID: 84106, WAV 파일: 2.motorcycle_horn_84106_1.wav, WAV 길이: 1.40s, JSON 길이: 5.40s, 오차: 4.00s
ID: 63502, WAV 파일: 2.motorcycle_horn_63502_1.wav, WAV 길이: 2.83s, JSON 길이: 6.83s,

> 주의
- duration 라벨의 경우, 원천데이터가 아닌 원시 데이터에 대한 정보이므로 활용에 유의해야 함

In [2]:
import os
import json
import re
import librosa
import pandas as pd

# 데이터 디렉토리 설정
data_dir = "/home/ubuntu/data"

# 파일 정보를 저장할 딕셔너리
wav_durations = {}
json_annotations = {}

# 고유 ID 추출을 위한 정규 표현식
id_pattern = re.compile(r'_(\d+)(?:_\d+)?$')  # 파일명에서 숫자 ID 추출

# 데이터 디렉토리 순회하며 파일 정보 수집
for root, _, files in os.walk(data_dir):
    for file in files:
        # WAV 파일 처리
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            try:
                # librosa로 파일 로드하여 오디오 데이터 및 샘플링 레이트 획득
                y, sr = librosa.load(file_path, sr=None, mono=False)

                # 파일 길이 계산 (초 단위)
                duration = librosa.get_duration(y=y, sr=sr)

                # 파일명에서 고유 ID 추출
                match = id_pattern.search(os.path.splitext(file)[0])
                if match:
                    file_id = match.group(1)
                    # 파일 정보 저장
                    wav_durations[file_id] = {
                        "wav_duration": duration,
                        "wav_path": file_path,
                        "wav_file_name": file
                    }
                else:
                    print(f"⚠️ WAV 파일에서 ID를 추출할 수 없습니다: {file_path}")

            except Exception as e:
                print(f"❌ WAV 파일 처리 중 오류 발생: {file_path}, 오류: {e}")

        # JSON 파일 처리
        elif file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data_json = json.load(f)

                    # 파일명에서 고유 ID 추출
                    match = id_pattern.search(os.path.splitext(file)[0])
                    if match:
                        file_id = match.group(1)

                        # annotations 섹션에서 각 annotation의 area 정보를 가져옴
                        annotations = data_json.get("annotations", [])
                        for annotation in annotations:
                            label_name = annotation.get("labelName", "")
                            # labelName에서 파일명 추출 및 ID 추출
                            label_match = id_pattern.search(os.path.splitext(label_name)[0])
                            if label_match:
                                label_id = label_match.group(1)
                                # area의 start와 end 값 가져오기
                                area = annotation.get("area", {})
                                start = area.get("start", None)
                                end = area.get("end", None)
                                if start is not None and end is not None:
                                    annotation_duration = end - start
                                    # 파일 정보 저장
                                    json_annotations[label_id] = {
                                        "annotation_duration": annotation_duration,
                                        "json_path": file_path,
                                        "json_file_name": file,
                                        "label_name": label_name
                                    }
                    else:
                        print(f"⚠️ JSON 파일에서 ID를 추출할 수 없습니다: {file_path}")

            except Exception as e:
                print(f"❌ JSON 파일 처리 중 오류 발생: {file_path}, 오류: {e}")

# WAV 파일과 JSON 파일 매칭하여 길이 비교
results = []

for file_id in json_annotations:
    if file_id in wav_durations:
        wav_info = wav_durations[file_id]
        annotation_info = json_annotations[file_id]

        wav_duration = wav_info["wav_duration"]
        annotation_duration = annotation_info["annotation_duration"]

        # 오차 계산 (초 단위)
        duration_error = abs(wav_duration - annotation_duration)
        # 허용 오차 범위 (예: 0.1초)
        tolerance = 0.1
        is_matching = duration_error <= tolerance

        results.append({
            "file_id": file_id,
            "wav_file_name": wav_info["wav_file_name"],
            "json_file_name": annotation_info["json_file_name"],
            "label_name": annotation_info["label_name"],
            "wav_duration": wav_duration,
            "annotation_duration": annotation_duration,
            "duration_error": duration_error,
            "is_matching": is_matching,
            "wav_path": wav_info["wav_path"],
            "json_path": annotation_info["json_path"]
        })
    else:
        print(f"⚠️ 매칭되는 WAV 파일이 없습니다: ID={file_id}, JSON 파일={annotation_info['json_file_name']}")

# 결과를 데이터프레임으로 변환
df_results = pd.DataFrame(results)

# 오차가 허용 범위를 벗어난 파일 필터링
mismatched_files = df_results[~df_results["is_matching"]]

print(f"\n총 파일 수: {len(df_results)}")
print(f"일치하는 파일 수: {len(df_results) - len(mismatched_files)}")
print(f"오차가 있는 파일 수: {len(mismatched_files)}")

# 오차가 있는 파일 목록 출력 (최대 20개만 표시)
if not mismatched_files.empty:
    print("\n⚠️ WAV 파일과 JSON 파일의 길이가 일치하지 않는 파일 목록 (최대 20개):")
    for idx, row in mismatched_files.head(20).iterrows():
        print(f"ID: {row['file_id']}, WAV 파일: {row['wav_file_name']}, WAV 길이: {row['wav_duration']:.2f}s, JSON 'annotations' 구간 길이: {row['annotation_duration']:.2f}s, 오차: {row['duration_error']:.2f}s")
else:
    print("\n✅ 모든 파일의 길이가 일치합니다!")

# 필요에 따라 CSV로 저장 (선택 사항)
# df_results.to_csv("wav_json_duration_comparison.csv", index=False)


총 파일 수: 18048
일치하는 파일 수: 18048
오차가 있는 파일 수: 0

✅ 모든 파일의 길이가 일치합니다!
